# Force Calibration Control Rig
## Arias Research Group
#### Seiya Ono

In [1]:
from ipywidgets import widgets, Layout, Button, Box, VBox
import subprocess, sys, serial, glob
# sudo pip install ipywidgets

In [2]:
# Helper Functions
def get_portlist():
    if sys.platform.startswith('win'):
        ports = ['COM' + str(i + 1) for i in range(256)]

    elif sys.platform.startswith('linux') or sys.platform.startswith('cygwin'):
        # this is to exclude your current terminal "/dev/tty"
        ports = glob.glob('/dev/tty[A-Za-z]*')

    elif sys.platform.startswith('darwin'):
        ports = glob.glob('/dev/tty.*')

    else:
        raise EnvironmentError('Unsupported platform')

    result = []
    for port in ports:
        try:
            s = serial.Serial(port)
            s.close()
            result.append(port)
        except (OSError, serial.SerialException):
            pass
    if len(result) < 1:
        print("No serial ports found. Reconnect and try again")
    return result

def select_port():
    portlist = get_portlist()

    # list the ports seen on the machine
    print("You have the following ports:")
    counter = 1
    for p in portlist:
        print("%i - %s"%(counter,p))
        counter = counter + 1

    # let the user choose the correct port
    index = int(input("Select the index of the port corresponding to the Arduino: \n"))
    while index > len(portlist):
        print("Please select the correct index")
        index = int(input("Select the index of the port corresponding to the Arduino: \n"))

    # connect to the port
    return portlist[index-1]

def serial_write(ser, data):
    if sys.platform.startswith('win'):
        ser.write([data ,])
    else:
        ser.write(data)

**Run this code block to connect with the serial port of the arduino:**

In [3]:
BAUD = 9600
arduino = serial.Serial(select_port(), BAUD)

You have the following ports:
1 - /dev/ttyACM0
Select the index of the port corresponding to the Arduino: 
1


In [4]:
# Buttons
left_bigskip_button  = Button(description='<<<', layout=Layout(flex='1 1 auto', width='auto'))
left_skip_button     = Button(description='<<', layout=Layout(flex='1 1 auto', width='auto'))
left_button          = Button(description='<', layout=Layout(flex='1 1 auto', width='auto'))
right_button         = Button(description='>', layout=Layout(flex='1 1 auto', width='auto'))
right_skip_button    = Button(description='>>', layout=Layout(flex='1 1 auto', width='auto'))
right_bigskip_button = Button(description='>>>', layout=Layout(flex='1 1 auto', width='auto'))

BASE_TICKS = 10
print_enable = True

def left_bigskip(b):
    if print_enable:
        print("Moving stage ", 10 * BASE_TICKS, " to the left")
    serial_write(arduino, b'a')
left_bigskip_button.on_click(left_bigskip)

def left_skip(b):
    if print_enable:
        print("Moving stage ", 5 * BASE_TICKS, " to the left")
    serial_write(arduino, b's')
left_skip_button.on_click(left_skip)
    
def left(b):
    if print_enable:
        print("Moving stage ", BASE_TICKS, " to the left")
    serial_write(arduino, b'd')
left_button.on_click(left)
    
def right(b):
    if print_enable:
        print("Moving stage ", BASE_TICKS, " to the right")
    serial_write(arduino, b'j')
right_button.on_click(right)
    
def right_skip(b):
    if print_enable:
        print("Moving stage ", 5 * BASE_TICKS, " to the right")
    serial_write(arduino, b'k')
right_skip_button.on_click(right_skip)
    
def right_bigskip(b):
    if print_enable:
        print("Moving stage ", 10 * BASE_TICKS, " to the right")
    serial_write(arduino, b'l')
right_bigskip_button.on_click(right_bigskip)

move = [left_bigskip_button, left_skip_button, left_button, 
        right_button, right_skip_button, right_bigskip_button]

box_layout = Layout(display='flex', flex_flow='row', align_items='stretch', width='100%')
box = Box(children=move, layout=box_layout)

In [5]:
# Start the UI
print_enable = False
display(box)

Box(children=(Button(description='<<<', layout=Layout(flex='1 1 auto', width='auto'), style=ButtonStyle()), Bu…

## Use the buttons to move the stage around

x axis calibration that has 0 - MAX_TICKS

want to control x axis

speed control - type in speed

maybe two point system to go back and forth between and set speed

things I need:

linear rotation, motor stepper angle, gear ratio